<a href="https://colab.research.google.com/github/NegativeGravity/Heuristic_Trader/blob/main/Temp_Heuristics_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
import random
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional, Literal
from numba import njit
from google.colab import drive

# ==========================================
# 1. CONFIGURATION & TYPES
# ==========================================
@dataclass(frozen=True)
class Config:
    """Immutable configuration for the trading system."""
    # Risk Management
    risk_per_trade: float = 0.01
    reward_risk_ratio: float = 2.0
    sl_min: float = 0.005
    sl_max: float = 0.04
    initial_capital: float = 1000.0
    commission: float = 0.0006

    # Genetic Algorithm
    pop_size: int = 100
    n_generations: int = 10000
    mutation_rate: int = 0.15
    max_conditions: int = 1
    min_trades: int = 30

    # WFA Settings
    train_window: int = 15000  # ~52 days (5m candles)
    test_window: int = 15000    # ~10 days

@dataclass
class Chromosome:
    """Represents a trading strategy candidate."""
    active_conds: np.ndarray
    feature_idxs: np.ndarray
    operators: np.ndarray
    threshold_quantiles: np.ndarray
    sl_gene: float

@dataclass
class StrategyResult:
    """Structured result of a strategy evaluation."""
    score: float
    trades: int
    win_rate: float
    equity: float
    rules: List[str]
    sl_pct: float
    tp_pct: float
    side: int # 1 for Long, -1 for Short

In [16]:
# ==========================================
# 2. NUMBA OPTIMIZED CORE (The Engine)
# ==========================================
@njit(fastmath=True)
def calculate_signals_numba(n_rows, max_conds, active_conds, feature_idxs,
                          operators, thresholds, feature_matrix):
    """
    Generates signals based on conditions. Pure function.
    """
    signal_mask = np.ones(n_rows, dtype=np.bool_)
    active_count = 0

    for i in range(max_conds):
        if active_conds[i] == 1:
            active_count += 1
            col_data = feature_matrix[:, feature_idxs[i]]
            thresh_val = thresholds[i]

            # Operator 0: < (Less Than), Operator 1: > (Greater Than)
            if operators[i] == 0:
                for r in range(n_rows):
                    if not (col_data[r] < thresh_val): signal_mask[r] = False
            else:
                for r in range(n_rows):
                    if not (col_data[r] > thresh_val): signal_mask[r] = False

    if active_count == 0:
        return np.zeros(n_rows, dtype=np.bool_), 0
    return signal_mask, active_count

@njit(fastmath=True)
def backtest_numba(close_prices: np.ndarray,
                   signals: np.ndarray,
                   side: int,  # 1 for Long, -1 for Short
                   sl_pct: float,
                   rr_ratio: float,
                   init_capital: float,
                   risk_per_trade: float,
                   commission: float) -> Tuple[int, int, float]:
    """
    Vectorized Backtest Engine supporting both Long and Short.
    """
    n_rows = len(close_prices)
    equity = init_capital
    wins = 0
    losses = 0
    tp_pct = sl_pct * rr_ratio

    i = 0
    while i < n_rows - 1:
        if signals[i]:
            entry_price = close_prices[i]

            # Position Sizing
            risk_amt = equity * risk_per_trade
            # Protect against zero division or negative prices
            if entry_price <= 0:
                i += 1
                continue

            position_size = risk_amt / (entry_price * sl_pct)

            outcome = 0
            for j in range(i + 1, n_rows):
                current_price = close_prices[j]

                # Calculate PnL % based on direction
                # Long: (Cur - Ent) / Ent
                # Short: (Ent - Cur) / Ent  => -1 * (Cur - Ent) / Ent
                raw_return = (current_price - entry_price) / entry_price
                pnl_pct = raw_return * side

                # Check TP
                if pnl_pct >= tp_pct:
                    gross_profit = position_size * entry_price * tp_pct
                    cost = (position_size * entry_price + position_size * current_price) * commission
                    equity += (gross_profit - cost)
                    wins += 1
                    outcome = 1
                    i = j
                    break

                # Check SL
                elif pnl_pct <= -sl_pct:
                    gross_loss = position_size * entry_price * sl_pct
                    cost = (position_size * entry_price + position_size * current_price) * commission
                    equity -= (gross_loss + cost)
                    losses += 1
                    outcome = -1
                    i = j
                    break

            if outcome == 0:
                i += 1 # Trade didn't close
        else:
            i += 1

    return wins, losses, equity

In [17]:
# ==========================================
# 3. GENETIC ALGORITHM LOGIC (SCORING 0-100)
# ==========================================
def create_random_chromosome(n_features: int, cfg: Config) -> Chromosome:
    return Chromosome(
        active_conds=np.random.randint(0, 2, size=cfg.max_conditions).astype(np.int64),
        feature_idxs=np.random.randint(0, n_features, size=cfg.max_conditions).astype(np.int64),
        operators=np.random.randint(0, 2, size=cfg.max_conditions).astype(np.int64),
        threshold_quantiles=np.random.uniform(0.05, 0.95, size=cfg.max_conditions),
        sl_gene=np.random.random()
    )

def decode_and_evaluate(chrom: Chromosome,
                       feature_matrix: np.ndarray,
                       close_prices: np.ndarray,
                       side: int,
                       cfg: Config) -> Tuple[float, int, float, float]:
    """
    Decodes chromosome -> Calculates Thresholds -> Signals -> Backtest -> Score (0-100)
    """
    # 1. Decode Thresholds
    actual_thresholds = np.zeros(cfg.max_conditions, dtype=np.float64)
    for k in range(cfg.max_conditions):
        if chrom.active_conds[k]:
            col_data = feature_matrix[:, chrom.feature_idxs[k]]
            actual_thresholds[k] = np.nanquantile(col_data, chrom.threshold_quantiles[k])

    # 2. Generate Signals
    signals, n_active = calculate_signals_numba(
        len(close_prices), cfg.max_conditions,
        chrom.active_conds, chrom.feature_idxs, chrom.operators,
        actual_thresholds, feature_matrix
    )

    if n_active == 0 or np.sum(signals) < 5: # Minimum valid trades to even score
        return 0.0, 0, 0.0, cfg.initial_capital

    # 3. Backtest
    sl_pct = cfg.sl_min + chrom.sl_gene * (cfg.sl_max - cfg.sl_min)
    wins, losses, end_equity = backtest_numba(
        close_prices, signals, side, sl_pct, cfg.reward_risk_ratio,
        cfg.initial_capital, cfg.risk_per_trade, cfg.commission
    )

    total_trades = wins + losses
    if total_trades < cfg.min_trades:
        # Penalize drastically if trades are insufficient but don't zero out if close
        return 0.0, total_trades, 0.0, end_equity

    net_profit = end_equity - cfg.initial_capital
    win_rate = wins / total_trades if total_trades > 0 else 0

    # 4. SCORING FUNCTION (Normalized 0-100)
    # Weights: 40% WinRate, 40% Profitability, 20% Activity

    # A. Win Rate Score (0 to 40)
    score_wr = win_rate * 40.0

    # B. Profit Score (0 to 40) - Capped at 100% ROI
    roi = net_profit / cfg.initial_capital
    score_profit = min(max(roi, 0.0), 1.0) * 40.0

    # C. Activity Score (0 to 20) - Capped at 100 trades
    score_activity = min(total_trades / 100.0, 1.0) * 20.0

    # Total Score
    final_score = score_wr + score_profit + score_activity

    # Severe penalty for negative profit
    if net_profit <= 0:
        final_score = final_score * 0.1 # Keep it low

    return final_score, total_trades, win_rate, end_equity

# ==========================================
# MODIFIED GA LOGIC (Clean Output)
# ==========================================
import copy

def evolve_population(df: pd.DataFrame,
                     side: int,
                     cfg: Config,
                     verbose: bool = False,
                     initial_population: List[Chromosome] = None) -> Tuple[Chromosome, List[str], List[Chromosome]]:
    """
    Runs GA. If initial_population is provided, it continues evolution from there.
    Returns: Best Chromosome, Feature Names, and the FINAL POPULATION.
    """
    feature_cols = [c for c in df.columns if c != 'close']
    feature_matrix = df[feature_cols].values.astype(np.float64)
    close_prices = df['close'].values.astype(np.float64)
    n_features = len(feature_cols)

    # --- 1. Load previous population if available ---
    if initial_population is not None:
        if verbose: print(f"   >> Loaded {len(initial_population)} strategies from previous window...")
        population = [copy.deepcopy(c) for c in initial_population]

        while len(population) < cfg.pop_size:
            population.append(create_random_chromosome(n_features, cfg))
        population = population[:cfg.pop_size]
    else:
        population = [create_random_chromosome(n_features, cfg) for _ in range(cfg.pop_size)]

    best_chrom = None
    best_score = -1.0

    # REMOVED: Table Header Print
    # if verbose:
    #     print(f"{'Gen':<5} {'Score':<10} {'Profit ($)':<12} {'Trades':<8} {'WinRate':<8}")
    #     print("-" * 50)

    for gen in range(cfg.n_generations):
        scores = []
        gen_best_score = -1
        # gen_best_stats = (0, 0, 0) # No longer needed for printing

        # Evaluation
        for chrom in population:
            score, tr, wr, eq = decode_and_evaluate(chrom, feature_matrix, close_prices, side, cfg)
            scores.append(score)

            # Global Best
            if score > best_score:
                best_score = score
                best_chrom = copy.deepcopy(chrom)

            # Generation Best (Logic kept for internal tracking, but printing removed)
            if score > gen_best_score:
                gen_best_score = score
                # gen_best_stats = (eq - cfg.initial_capital, tr, wr)

        # REMOVED: Table Row Print
        # if verbose and (gen % 10 == 0 or gen == cfg.n_generations - 1):
        #      profit, tr, wr = gen_best_stats
        #      print(f"{gen:<5} {gen_best_score:<10.1f} {profit:<12.2f} {tr:<8} {wr*100:<7.1f}%")

        # Selection & Mutation (Standard GA)
        next_pop = [best_chrom] # Elitism

        while len(next_pop) < cfg.pop_size:
            p1, p2 = random.sample(range(cfg.pop_size), 2)
            parent = population[p1] if scores[p1] > scores[p2] else population[p2]

            child = copy.deepcopy(parent)

            if random.random() < cfg.mutation_rate:
                child.active_conds[random.randint(0, cfg.max_conditions - 1)] = 1 - child.active_conds[random.randint(0, cfg.max_conditions - 1)]
            if random.random() < cfg.mutation_rate:
                child.feature_idxs[random.randint(0, cfg.max_conditions - 1)] = random.randint(0, n_features - 1)
            if random.random() < cfg.mutation_rate:
                idx = random.randint(0, cfg.max_conditions - 1)
                child.threshold_quantiles[idx] = np.clip(child.threshold_quantiles[idx] + np.random.normal(0, 0.1), 0.05, 0.95)

            next_pop.append(child)

        population = next_pop

    return best_chrom, feature_cols, population

In [18]:
# ==========================================
# 4. WALK-FORWARD ANALYSIS (EXPANDING WINDOW)
# ==========================================
# ==========================================
# MODIFIED WFA (Passes Population Forward)
# ==========================================
def run_walk_forward_optimization(df: pd.DataFrame,
                                side: int,
                                cfg: Config) -> Tuple[List[Dict], List[float]]:
    """
    Performs Anchored Walk-Forward Analysis with Population Persistence.
    0-15k -> Train -> Test 15-30k ->
    0-30k -> Train (Start with Pop from 0-15k) -> Test 30-45k -> ...
    """
    n_rows = len(df)
    current_split = cfg.train_window

    oos_trades = []
    equity_curve = [cfg.initial_capital]
    running_capital = cfg.initial_capital

    # --- STATE VARIABLE FOR POPULATION ---
    current_population = None

    side_name = "LONG" if side == 1 else "SHORT"
    print(f"\n🚀 Starting PERSISTENT Walk-Forward for {side_name}")
    print(f"   Initial Train Size: {cfg.train_window} | Test Step: {cfg.test_window}")

    window_count = 0

    while current_split + cfg.test_window <= n_rows:
        window_count += 1

        # 1. Define Windows
        train_start = 0
        train_end = current_split
        test_start = current_split
        test_end = current_split + cfg.test_window

        df_train = df.iloc[train_start:train_end].copy()
        df_test = df.iloc[test_start:test_end].copy()

        # 2. Optimize (Transferring Knowledge)
        print(f"\n🔄 Training Window {window_count}: Rows 0 to {train_end}")
        if current_population is not None:
             print(f"   >> Injecting {len(current_population)} strategies from previous window...")

        # --- CALL GENETIC ALGORITHM WITH PREVIOUS POPULATION ---
        best_chrom, feats, end_population = evolve_population(
            df_train, side, cfg, verbose=False, initial_population=current_population
        )

        # Save population for the NEXT larger window
        current_population = end_population

        # 3. Reporting & Testing (Standard Logic)
        feature_matrix_train = df_train[feats].values.astype(np.float64)
        score_train, tr_train, wr_train, eq_train = decode_and_evaluate(
            best_chrom, feature_matrix_train, df_train['close'].values.astype(np.float64), side, cfg
        )

        sl_pct = cfg.sl_min + best_chrom.sl_gene * (cfg.sl_max - cfg.sl_min)

        # Validation on OOS
        feature_matrix_test = df_test[feats].values.astype(np.float64)
        actual_thresholds = np.zeros(cfg.max_conditions)
        for k in range(cfg.max_conditions):
            if best_chrom.active_conds[k]:
                idx = best_chrom.feature_idxs[k]
                actual_thresholds[k] = np.nanquantile(feature_matrix_train[:, idx], best_chrom.threshold_quantiles[k])

        signals_test, _ = calculate_signals_numba(
            len(df_test), cfg.max_conditions,
            best_chrom.active_conds, best_chrom.feature_idxs, best_chrom.operators,
            actual_thresholds, feature_matrix_test
        )

        wins, losses, segment_end_eq = backtest_numba(
            df_test['close'].values.astype(np.float64), signals_test, side, sl_pct, cfg.reward_risk_ratio,
            cfg.initial_capital, cfg.risk_per_trade, cfg.commission
        )

        segment_profit = segment_end_eq - cfg.initial_capital
        running_capital += segment_profit
        equity_curve.append(running_capital)

        print(f"   🧪 OOS Result: Profit ${segment_profit:.2f} | WR: {(wins/(wins+losses)*100) if (wins+losses)>0 else 0:.1f}%")

        oos_trades.append({
            'window': window_count,
            'profit': segment_profit,
            'equity': running_capital
        })

        current_split += cfg.test_window

    return oos_trades, equity_curve

In [19]:
# ==========================================
# 5. DATA UTILS & REPORTING
# ==========================================
def load_data(filepath: str) -> pd.DataFrame:
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, parse_dates=True, index_col=0)
    df.columns = [c.lower() for c in df.columns]

    # Cleaning
    drops = ['open', 'high', 'low', 'volume']
    existing_drops = [c for c in drops if c in df.columns and c != 'close']
    df.drop(columns=existing_drops, inplace=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    # Cast to float32/64
    for col in df.columns:
        df[col] = df[col].astype(np.float32)

    return df

def align_datasets(df_train: pd.DataFrame, df_test: pd.DataFrame) -> pd.DataFrame:
    """Ensures test set has exact same columns as train set."""
    feature_cols = [c for c in df_train.columns if c != 'close']
    for c in feature_cols:
        if c not in df_test.columns:
            df_test[c] = 0.0
    return df_test[feature_cols + ['close']].copy()

def print_strategy_report(title: str, chrom: Chromosome, feats: List[str],
                         train_matrix: np.ndarray, sl_pct: float, rr: float):
    print("\n" + "="*50)
    print(f"📜 {title}")
    print("="*50)
    print(f"Risk Settings: SL {sl_pct*100:.2f}% | TP {sl_pct*rr*100:.2f}%")
    print("Entry Conditions:")

    for k in range(len(chrom.active_conds)):
        if chrom.active_conds[k]:
            idx = chrom.feature_idxs[k]
            # Decode threshold value for display
            val = np.nanquantile(train_matrix[:, idx], chrom.threshold_quantiles[k])
            op = ">" if chrom.operators[k] == 1 else "<"
            print(f"  • {feats[idx]} {op} {val:.5f} (q={chrom.threshold_quantiles[k]:.2f})")
    print("-" * 50)

In [20]:
# ==========================================
# 6. MAIN EXECUTION (FULL EXPANDING LOGIC)
# ==========================================

def run_full_analysis(side_name: str, side_val: int, df_train, df_test, cfg):
    print(f"\n" + "█"*60)
    print(f"   STARTING ANALYSIS FOR: {side_name} (Side={side_val})")
    print("█"*60)

    # --- PART A: Walk-Forward Analysis ---
    # Using the new Expanding Window function
    print(f"\n📊 Phase 1: Expanding Walk-Forward Analysis...")
    logs, equity_curve = run_walk_forward_optimization(df_train, side_val, cfg)

    wfa_profit = equity_curve[-1] - cfg.initial_capital
    print(f"\n   >> Cumulative WFA Profit: ${wfa_profit:.2f}")

    # --- PART B: Final Training ---
    print(f"\n🔬 Phase 2: Final Training (Using ALL Training Data)...")
    # For Expanding Window strategy, we train on the FULL dataset before hitting the Held-Out test
    # Previously we sliced [-window:], now we use everything.
    df_full_train = df_train.copy()

    # We show the progress table here only (verbose=True) as requested
    best_chrom, feats = evolve_population(df_full_train, side_val, cfg, verbose=True)

    # Decode Final Stats
    sl_pct = cfg.sl_min + best_chrom.sl_gene * (cfg.sl_max - cfg.sl_min)
    tp_pct = sl_pct * cfg.reward_risk_ratio

    # --- PART C: Held-Out Test ---
    print(f"\n🧪 Phase 3: Testing on Held-Out Data...")

    # Prepare Test Data
    feature_matrix_test = df_test[feats].values.astype(np.float64)
    close_test = df_test['close'].values.astype(np.float64)
    feature_matrix_train = df_full_train[feats].values.astype(np.float64)

    # Decode Thresholds
    actual_thresholds = np.zeros(cfg.max_conditions)
    print("\n📜 SELECTED RULES (From Full Training Data):")
    for k in range(cfg.max_conditions):
        if best_chrom.active_conds[k]:
            idx = best_chrom.feature_idxs[k]
            val = np.nanquantile(feature_matrix_train[:, idx], best_chrom.threshold_quantiles[k])
            actual_thresholds[k] = val
            op = ">" if best_chrom.operators[k] == 1 else "<"
            print(f"   • {feats[idx]} {op} {val:.5f}")

    # Backtest
    signals_final, _ = calculate_signals_numba(
        len(df_test), cfg.max_conditions,
        best_chrom.active_conds, best_chrom.feature_idxs, best_chrom.operators,
        actual_thresholds, feature_matrix_test
    )

    wins, losses, end_equity = backtest_numba(
        close_test, signals_final, side_val, sl_pct, cfg.reward_risk_ratio,
        cfg.initial_capital, cfg.risk_per_trade, cfg.commission
    )

    trades = wins + losses
    wr = (wins / trades) * 100 if trades > 0 else 0.0
    net_profit = end_equity - cfg.initial_capital

    print("-" * 40)
    print(f"🏁 FINAL HELD-OUT RESULT ({side_name}):")
    print(f"   Net Profit:        ${net_profit:.2f}")
    print(f"   Trades:            {trades}")
    print(f"   Win Rate:          {wr:.2f}%")
    print(f"   Risk Settings:     SL={sl_pct*100:.2f}% | TP={tp_pct*100:.2f}%")
    print("-" * 40)

    return net_profit

In [ ]:
if __name__ == "__main__":
    try:
        drive.mount('/content/drive/')
    except: pass

    # Paths
    TRAIN_PATH = "/content/drive/MyDrive/Heuristics_Project/Phase 2/eth_5m_with_features.csv"
    TEST_PATH  = "/content/drive/MyDrive/Heuristics_Project/Phase 2/eth_5m_with_features_test.csv"

    try:
        df_train_full = load_data(TRAIN_PATH)
        df_test_heldout = load_data(TEST_PATH)
        df_test_heldout = align_datasets(df_train_full, df_test_heldout)
        cfg = Config()

        # Run Long
        profit_long = run_full_analysis("LONG (BUY)", 1, df_train_full, df_test_heldout, cfg)

        # Run Short
        profit_short = run_full_analysis("SHORT (SELL)", -1, df_train_full, df_test_heldout, cfg)

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Loading /content/drive/MyDrive/Heuristics_Project/Phase 2/eth_5m_with_features.csv...
Loading /content/drive/MyDrive/Heuristics_Project/Phase 2/eth_5m_with_features_test.csv...

████████████████████████████████████████████████████████████
   STARTING ANALYSIS FOR: LONG (BUY) (Side=1)
████████████████████████████████████████████████████████████

📊 Phase 1: Expanding Walk-Forward Analysis...

🚀 Starting PERSISTENT Walk-Forward for LONG
   Initial Train Size: 15000 | Test Step: 15000

🔄 Training Window 1: Rows 0 to 15000
   🧪 OOS Result: Profit $0.00 | WR: 0.0%

🔄 Training Window 2: Rows 0 to 30000
   >> Injecting 100 strategies from previous window...
   🧪 OOS Result: Profit $-132.24 | WR: 32.7%

   >> Cumulative WFA Profit: $-132.24

🔬 Phase 2: Final Training (Using ALL Training Data)...
